# Xpand IT DS Challenge

## Configuration

### Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Configure pyplot and seaborn

In [2]:
# If set, put axes ticks and the gridlines below the plots
plt.rc("axes", axisbelow=True)

# [NOTE: seaborn uses matplotlib's rc (runtime configuration) and, therefore, there is
# no need to configure Seaborn's rc in particular.]

### Define a DataFrame Styling Function

In [3]:
def style_df(df, caption=""):
    return (df.style
        .set_caption(caption)
        .set_table_styles([
            {
                "selector": "caption",
                "props": [
                    ("width", "100%"),
                    ("font-size", "16px"),
                    ("font-weight", "bold"),
                    ("font-style", "italic"),
                    ("text-align", "left"),
                    ("padding-top", "15pt"),
                    ("padding-bottom", "5pt"),
                    ("padding-left", "15pt"),
                ]
            },
            {
                "selector": "td",
                "props": [
                    ("padding-bottom", "10pt"),
                    ] 
            }
        ])
    )

## Business Analysis

### What is the Dow Jones Index?
The Dow Jones Index, also called **Dow Jones Industrial Average**, is a stock market index that rates the combination of $30$ prominent companies of the United States of America. It is based on the values of the stock shares of each of these $30$ companies. Its computation formula corresponds to

$$
\mathrm{DJIA} = \frac{\sum_{i} p_i}{d}\text{ ,}
$$

where $p_i$ is the price of a single stock share of the $i$-th company and $d$ is the so-called **Dow Divisor**. This divisor is adjusted in case of stock splits, spinoffs or similar structural changes. On [$26^{\mathrm{th}}$ of February of $2024$](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Computation), its value corresponded to $0.15221633137872$.

The list of $30$ companies accounted by the index may actually change as these cease to be prominent, underperforming when compared to those companies "below" them. As of February of $26^{\mathrm{th}}$ of February of $2024$, the list has changed $58$ times. To ensure that the Dow Jones Index remains continuous in face of such change the Dow Divisor $d$ is updated accordingly.

### How does the issued data compares with the Dow Jones Index?

The issued data does not correspond to the Dow Jones Index, but contains all the necessary data for computing it except the Dow Divisor (although out-of-the-scope for this project, and no table has been found, the present author believes that a more thorough search could be successful in retrieving Dow Divisor's historic values). The data is comprised by the stock share values of the $30$ prominent companies at the different weeks of the first two quarters of $2011$, plus some additional and useful info, such as **volume of exchanged  shares**, and **dividend yields**. 

* The **volume of exchanged shares** may describe the liquidity of the market (high values would make buying or selling shares to not significantly affect the stock's price), the stock price "momentum" (spikes in exchanged volume might start new trends) as well as the investor "sentiment" (high values indicate high interest and confidence in the stock);

* Companies may pay at certain times an amount to a shareholder based on the amount of shares held and a stipulated factor: the **dividend yield**. This corresponds to the ratio between the value of the dividend payment per stock share and the value of that same stock share. High values are usually associated with a high profitability and management confidence of the company, promoting further reinvestment by the shareholders, which could then increase the stock value.

[***NOTE:*** if the so-called ex-dividend dates were issued together with the data, one could have actually considered the dividend return to be part of the profit of the investor in the current problem. Since they were not, the computed profit was solely based on the relative change of the stock prices.]

All of these data are important for building a model that could reliably predict the evolution of the stock prices.

## Data Understanding

### Import the data

In [4]:
data = pd.read_csv("data/dow_jones_index.data")
display(style_df(df=data.head(), caption="Raw data"))

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,$15.82,$16.72,$15.78,$16.42,239655616,3.792670,nan,nan,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,$16.71,$16.71,$15.64,$15.97,242963398,-4.428490,1.380223,239655616.000000,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,$16.19,$16.38,$15.60,$15.79,138428495,-2.470660,-43.024959,242963398.000000,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,$15.87,$16.63,$15.82,$16.13,151379173,1.638310,9.355500,138428495.000000,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,$16.18,$17.39,$16.18,$17.14,154387761,5.933250,1.987452,151379173.000000,$17.33,$17.37,0.230814,97,0.175029


### Understand, transform, create and drop data

* **Column `quarter`:** 

    * It has the year quarter numbers (either $1$ or $2$) in the respective weeks;

    * **The column is to be used as <ins>feature</ins>.**

* **Column `stock`:**
    * It has the acronyms that identify stock's company.

    * **The column is to be used for company-wise data splitting (value "MMM" for company 3M, "AXP" for American Express, ...);**

    * [***Note:*** since a model is to be created per company, the data needs to be split by company.]

* **Column `date`:** 

    * It has the dates for the closing of the stock exchange in the respective weeks. **One should not directly use dates as features since they comprise a very large number of categories, whose some part may be "seen" during training but not during testing, inducing overfitting. For instance, by training with the data of the first quarter and testing with the data of the second, if the dates are used as features, the training cases would be supported but the testing ones not at all and extrapolation would occur.** However, if elements of the dates are seen in both batches (usually cyclic elements, such as the weekday, month, etc.) these could be used as features in place of the dates. In the current problem, the range of dates would not allow a proper usage of the common date cyclic elements. **Still, one could think of using an "artificial" cycling element, such as the ratio between the day number and the total number days of the month. For instance, the $8^{\mathrm{th}}$ day of a 31 days month would be associated with a ratio $8/31 \times 100 \,\%\approx 25.8\%$. Let `percent_month` be the resultant new column;**

    * To use the date as a simpler data identifier, one could tranform it into year's week number. Let the new column be `week_num`;

    * **New column `week_num` is to be used as <ins>data identifier (row index)</ins>.**

    * **New column `percent_month` is to be used as <ins>feature</ins>.**

In [5]:
# Tranform column "date" into pandas Timestamp
data["date"] = pd.to_datetime(data["date"], format="%m/%d/%Y")

# Add column with percentage of the month associated with the dates
data["percent_month"] = data["date"].dt.strftime(date_format="%m/%d/%Y").apply(
    lambda x: pd.Period(x).days_in_month
)

# Add column with year's week numbers associated with the dates
data["week_num"] = data["date"].dt.isocalendar().week

# Drop column "date"
data = data.drop(columns="date")

* **Columns `open`, `high`, `low`, `close` and `percent_change_price`:**

    * The first four have the value per share of the stocks at the opening, best day, worse day and closing of the market at that week, respectively. They are formatted as strings and prefixed by the american dollar "$\$$". **To properly handle them, they should be converted to floats;**

    * Also, since column `percent_change_price` (relative difference (in percentage) between closing and opening values) are simply obtained from  `open` and `close` one of the columns would be redundant;

    * **Column `close` is to be dropped;**

    * **Columns `open`, `high`, `low` and `percent_change_price` are to be used as <ins>features</ins>.**

In [6]:
# Drop column "next_weeks_close"
data = data.drop(columns="close")

# Re-format columns "open", "high" and "low", to disregard the dollar sign
for col in ["open", "high", "low"]:
    data[col] = data[col].apply(lambda x: float(x.removeprefix("$")))

* **Columns `volume`, `percent_change_volume_over_last_wk` and `previous_weeks_volume`:**

    * Since `percent_change_volume_over_last_wk` (relative difference (in percentage) of exchanged volume of stock shares between current and previous week) are obtained from `previous_weeks_volume` and `volume`, one should disregard one of the columns. Arbitrarily, it was decided for `previous_weeks_volume` to be that one;

    * Furthermore, since multiple previous weeks besides the current one are to be used as feature data for the prediction of the following week, one should disregard the columns `volume` of the previous weeks, since they may be computed from the column `volume` of the current week together with the columns `percent_change_volume_over_last_wk` of the current and previous weeks, making them redundant.

    * **Column `previous_weeks_volume` is to be dropped;**

    * **Column `percent_change_volume_over_last_wk` is to be used as <ins>feature</ins>.**

    * **Column `volume` is to be used as <ins>feature of the current week</ins>;**

In [ ]:
# Drop column "next_weeks_close"
data = data.drop(columns="previous_weeks_volume")

* **Columns `next_weeks_open`, `next_weeks_close` and `percent_change_next_weeks_price`:**

    * Since `percent_change_next_weeks_price` (relative difference (in percentage) between closing and opening stock values of the future week) are obtained from `next_weeks_open` and `next_weeks_close`, one of them should to be dropped;

    * Also, since multiple previous weeks besides the current one are to be used as feature data for the prediction of the following week, one should disregard the columns `next_weeks_open` of the previous weeks, since they coincide with the columns `open` of the following contiguous ones making them redundant.

    * Note that the values of the columns `next_weeks_open` and `next_weeks_close`are formatted as strings and prefixed by the american dollar "$\$$". They should be re-formatted as floats.

    * **Column `next_weeks_close` is to be dropped;**

    * **Column `next_weeks_open` is to be used as <ins>feature of the current week</ins>;**

    * **Column `percent_change_next_weeks_price` is to be used as <ins>target</ins>.**

In [9]:
# Drop column "next_weeks_close"
data = data.drop(columns="next_weeks_close")

# Re-format column "next_weeks_open" to disregard the dollar sign
data["next_weeks_open"] = data["next_weeks_open"].apply(
    lambda x: float(x.removeprefix("$"))
)

* **Columns `days_to_next_dividend` and `percent_return_next_dividend`:**

    *  The former is the number of days from current week to the day of the next dividend payment. The latter is the ratio (in percentage) between the value of the dividend payment per stock share and the value of that same stock share, based on the state of the current week;
    
    * **The columns are to be used as <ins>features</ins>.**

In [14]:
# ---> Display the new DataFrame
# Reorder DataFrame's columns
data = data[[
    "stock", "week_num", "quarter", "percent_month", "open", "high", "low",
    "percent_change_price",	"volume", "percent_change_volume_over_last_wk",
    "next_weeks_open", "days_to_next_dividend", "percent_return_next_dividend", "percent_change_next_weeks_price"]]

display(style_df(df=data.head(), caption="Raw data"))

,stock,week_num,quarter,percent_month,open,high,low,percent_change_price,volume,percent_change_volume_over_last_wk,next_weeks_open,days_to_next_dividend,percent_return_next_dividend,percent_change_next_weeks_price
0,AA,1,1,31,15.820000,16.720000,15.780000,3.792670,239655616,nan,16.710000,26,0.182704,-4.428490
1,AA,2,1,31,16.710000,16.710000,15.640000,-4.428490,242963398,1.380223,16.190000,19,0.187852,-2.470660
2,AA,3,1,31,16.190000,16.380000,15.600000,-2.470660,138428495,-43.024959,15.870000,12,0.189994,1.638310
3,AA,4,1,31,15.870000,16.630000,15.820000,1.638310,151379173,9.355500,16.180000,5,0.185989,5.933250
4,AA,5,1,28,16.180000,17.390000,16.180000,5.933250,154387761,1.987452,17.330000,97,0.175029,0.230814


### Exploratory data analysis

In [ ]:

# ---> Define cuatom histogram plotting function
def hisplot_custom(
    df,
    col,
    # Data type to which the column is to be transformed
    col_dtype=float,
    stat="count",
    bins=20,
    shrink=1.0,
    bar_label=True,
    bar_label_rotation=0,
    xticklabels_rotation=0,
):
    
    # Initialise figure and axes
    plt.figure(figsize=(9, 6))
    ax = plt.axes()
    plt.title(
        "Occurrence" + 
        (" fractions " if stat == "percent" else " counts ") +
        "per value" +
        (" range " if stat == "percent" else " ") + f"in column \"{col}\"",
        pad=20
    )


    # Define bar plot
    sns.histplot(
        ax=ax,
        data=df.astype({col: col_dtype}),
        x=col,
        stat=stat,
        bins=bins,
        shrink=shrink,
        edgecolor="black",
        alpha=1,
        color="blue",
        linewidth=1.0,
    )

    if bar_label == True:
        # Plot bars' values
        for container in ax.containers:
            ax.bar_label(
                container=container,
                padding=2,
                fmt=r"${:.2f}\,\%$" if stat == "percent" else r"${:.0f}$",
                size=9,
                rotation=bar_label_rotation
            )

    # Define axes labels                                
    ax.set_xlabel(col, fontdict={"fontsize": 10})
    ax.set_ylabel(
        r"Fraction $[\%]$" if stat == "percent" else r"Count",
        fontdict={"fontsize": 10}
    )

    # Define axes ticks
    ax.tick_params(axis="x", rotation=xticklabels_rotation)

    # Define axes ranges
    ax.set_ylim(top=1.1 * ax.get_ylim()[1])

    # Enable axes' minor ticks
    ax.minorticks_on()

    # Define grid
    ax.grid(
        visible=True,
        which="major",
        color="lightgray",
        linestyle="solid",
        linewidth=0.5
    )
    ax.grid(
        visible=True,
        which="minor",
        color="lightgray",
        linestyle="dotted",
        linewidth=0.5
    )
    
    # Show plot
    plt.show() 

In [ ]:
hisplot_custom(
    df=data,
    col="percent_change_volume_over_last_wk",
    stat="percent",
    bins=50,
    bar_label_rotation=90
)

###  Handling missing values

In [ ]:
# ---> Get missing value counts

N_na = data.isna().sum()
display(style_df(df=N_na.to_frame(name="N_na"), caption="Counts of missing values per column"))

Solely column `percent_change_volume_over_last_wk` has missing values, and, coincidently, of number $30$, which is also the number of companies regarded by the Dow Jones index. One may already suspect that such missing values are associated with the first week, as this one might not have volume data associated with the past.

Let one display the unique dates associated with the missing values to assert this suspicion.

In [ ]:
# ---> Get dates associated with the missing values

na_date_unique = data[data.isna().any(axis=1)]["date"].unique()
display(style_df(
    pd.DataFrame({"date": [na_date_unique]}),
    caption="Unique dates associated with the missing values"
))


All of the dates associated with the missing values correspond to the first week of the first quarter, as suspected. Assuming that the values of the column `percent_change_volume_over_last_wk` do not drastically change in time, it would be reasonable to fill the missing values with the ones of the respective companies at the following week.  

### Prepare the data ........

## Modelling
In this phase, your main goal is to develop and describe your approach to the solution of the problem. Some guidelines to help you:
* What metrics will you use to evaluate your solutions?
* What are some algorithms that can lead to good results? And why?
* Describe in detail your thought process during the development of your solution.
* Present your results.


-----


In [ ]:
#add here your modelling code

-----
*Add here your modelling results and conclusions (max. 200 words)*

-----

## Conclusions
In the conclusions, you should enumerate the results you got after completing the challenge.
* How good do you consider your results? 
* What are some factors that would contribute to get better results?
* What are some advantages and disadvantages of your solution?
* What can be done as future work to improve your results?


-----
*Add here your final conclusions (max. 400 words)*

-----

#### Feedback

-----
*Add here your thoughts and feedback regarding this challenge.*

-----

To submit your solution you should e-mail us this notebook in response to the e-mail you initially received with the challenge.